In [1]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

import joblib

# Veriyi yükle
df = pd.read_csv('veri.csv')

# Veriyi incele
print(df.head())


  Ev Tipi          Konum  Oda Sayısı  Konaklayabilecek Kişi Sayısı  Havuz  \
0   villa  şehir merkezi           1                             1      0   
1     oda  şehir merkezi           2                             6      0   
2   villa  şehir merkezi           4                             6      0   
3   villa  şehir merkezi           5                             1      1   
4     oda         kırsal           3                             3      1   

   WiFi  Kahvaltı  Fiyat  
0     0         1    172  
1     0         0    178  
2     0         0    229  
3     1         1    291  
4     0         0    250  


In [2]:
import pandas as pd

# Veriyi yükle
df = pd.read_csv('veri.csv')

# Veriyi incele
df.head()


,Ev Tipi,Konum,Oda Sayısı,Konaklayabilecek Kişi Sayısı,Havuz,WiFi,Kahvaltı,Fiyat
0,villa,şehir merkezi,1,1,0,0,1,172
1,oda,şehir merkezi,2,6,0,0,0,178
2,villa,şehir merkezi,4,6,0,0,0,229
3,villa,şehir merkezi,5,1,1,1,1,291
4,oda,kırsal,3,3,1,0,0,250


In [3]:
# Özellikler ve hedef değişkeni ayırma
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']

# Kategorik ve sayısal sütunları belirleme
categorical_columns = ['Ev Tipi', 'Konum']
numerical_columns = ['Oda Sayısı', 'Konaklayabilecek Kişi Sayısı', 'Havuz', 'WiFi', 'Kahvaltı']

# Kategorik değişkenler için OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)


In [4]:
# Eğitim ve test setlerini ayırma
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Boyutları kontrol et
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((800, 7), (200, 7), (800,), (200,))

In [ ]:
# Algoritmaların listesi
from xgboost import XGBRegressor


algorithms = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),


    "SVR": SVR(),
    "KNN": KNeighborsRegressor()
}

# Algoritmaları dene ve RMSE değerlerini hesapla
results = {}

for name, model in algorithms.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = rmse
    print(f"{name}: RMSE = {rmse}")
    
    
    # En iyi modeli belirle
best_model_name = min(results, key=results.get)
best_model = algorithms[best_model_name]

print(f"\nEn iyi model: {best_model_name} (RMSE: {results[best_model_name]})")


# En iyi modeli tekrar eğit ve kaydet
final_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])
final_pipeline.fit(X_train, y_train)

# Modeli kaydet

from joblib import dump

dump(final_pipeline, 'eniyi.joblib')

print("En iyi model başarıyla 'eniyi.joblib' dosyasına kaydedildi!")




NameError: name 'CatBoostRegressor' is not defined